# **BGP**

In [1]:
import networkx as nx
import matplotlib
matplotlib.use('Agg') # Use a non-GUI backend
import matplotlib.pyplot as plt
import time

def draw_as_graph(graph, pos, title):
    """Helper function to draw the AS-level graph."""
    plt.figure(figsize=(10, 6))
    nx.draw(graph, pos, with_labels=True, node_color='lightgreen', node_size=3000,
            font_size=18, font_weight='bold')
    plt.title(title)

    filename = "bgp_topology.png"
    plt.savefig(filename)
    print(f"\n*** Graph saved to {filename} ***")
    plt.close()

def simulate_bgp():
    """Simulates the Border Gateway Protocol (BGP) as a Path Vector protocol."""

    print("--- Simulating BGP (Path Vector) ---")

    ases = [100, 200, 300, 400]
    links = [(100, 200), (200, 300), (300, 400), (400, 100), (200, 400)]

    G = nx.Graph()
    G.add_edges_from(links)
    pos = nx.circular_layout(G)
    draw_as_graph(G, pos, "BGP AS-Level Topology")

    prefixes = {
        100: '10.1.0.0/16',
        200: '20.2.0.0/16',
        300: '30.3.0.0/16',
        400: '40.4.0.0/16'
    }

    rib = {asn: {} for asn in ases}

    for asn, prefix in prefixes.items():
        rib[asn][prefix] = {
            'as_path': [asn],
            'next_hop': 'self'
        }

    print(f"ASes: {ases}")
    print(f"AS Links: {links}\n")
    print("--- Simulating BGP UPDATEs ---")

    round_num = 0
    while True:
        round_num += 1
        print(f"--- BGP CONVERGENCE ROUND {round_num} ---")

        changed = False
        rib_snapshot = {asn: table.copy() for asn, table in rib.items()}

        for u_asn in ases:
            for v_asn in G.neighbors(u_asn):
                for prefix, info in rib_snapshot[u_asn].items():
                    if v_asn in info['as_path']:
                        continue

                    new_path = [u_asn] + info['as_path']

                    if prefix not in rib[v_asn]:
                        rib[v_asn][prefix] = {
                            'as_path': new_path,
                            'next_hop': u_asn
                        }
                        changed = True
                    else:
                        current_path_len = len(rib[v_asn][prefix]['as_path'])
                        new_path_len = len(new_path)

                        if new_path_len < current_path_len:
                            rib[v_asn][prefix] = {
                                'as_path': new_path,
                                'next_hop': u_asn
                            }
                            changed = True

        if not changed:
            print(f"\n*** BGP CONVERGENCE REACHED in {round_num} rounds. ***\n")
            break

        if round_num > 5:
            print("Reached max rounds, stopping.")
            break

        time.sleep(1)

    print("--- FINAL BGP ROUTING TABLES (RIBs) ---")
    for asn in ases:
        print(f"AS {asn}'s RIB:")
        print(f"  {'Prefix':<15} | {'Next Hop AS':<12} | {'AS_PATH':<20}")
        print("  " + "-"*50)
        for prefix, info in sorted(rib[asn].items()):
            path_str = " -> ".join(map(str, info['as_path']))
            print(f"  {prefix:<15} | {info['next_hop']:<12} | {path_str}")
        print()

if __name__ == "__main__":
    simulate_bgp()

--- Simulating BGP (Path Vector) ---

*** Graph saved to bgp_topology.png ***
ASes: [100, 200, 300, 400]
AS Links: [(100, 200), (200, 300), (300, 400), (400, 100), (200, 400)]

--- Simulating BGP UPDATEs ---
--- BGP CONVERGENCE ROUND 1 ---
--- BGP CONVERGENCE ROUND 2 ---
--- BGP CONVERGENCE ROUND 3 ---

*** BGP CONVERGENCE REACHED in 3 rounds. ***

--- FINAL BGP ROUTING TABLES (RIBs) ---
AS 100's RIB:
  Prefix          | Next Hop AS  | AS_PATH             
  --------------------------------------------------
  10.1.0.0/16     | self         | 100
  20.2.0.0/16     | 200          | 200 -> 200
  30.3.0.0/16     | 200          | 200 -> 300 -> 300
  40.4.0.0/16     | 400          | 400 -> 400

AS 200's RIB:
  Prefix          | Next Hop AS  | AS_PATH             
  --------------------------------------------------
  10.1.0.0/16     | 100          | 100 -> 100
  20.2.0.0/16     | self         | 200
  30.3.0.0/16     | 300          | 300 -> 300
  40.4.0.0/16     | 400          | 400 -> 400



# **ISIS**

In [2]:
import networkx as nx
import matplotlib
matplotlib.use('Agg') # Use a non-GUI backend
import matplotlib.pyplot as plt
import heapq

def draw_graph_with_costs(graph, pos, title):
    """Helper function to draw the network graph with link costs."""
    plt.figure(figsize=(12, 8))
    labels = nx.get_edge_attributes(graph, 'weight')
    nx.draw(graph, pos, with_labels=True, node_color='lightcoral', node_size=2500, font_size=16, font_weight='bold')
    nx.draw_networkx_edge_labels(graph, pos, edge_labels=labels, font_color='blue')
    plt.title(title)

    filename = "isis_topology.png"
    plt.savefig(filename)
    print(f"\n*** Graph saved to {filename} ***")
    plt.close()

def dijkstra(graph, start_node):
    """Implements Dijkstra's algorithm to find shortest paths."""

    pq = [(0, start_node, None)]
    distances = {node: float('inf') for node in graph.nodes}
    distances[start_node] = 0
    predecessors = {node: None for node in graph.nodes}
    visited = set()

    while pq:
        cost, node, prev = heapq.heappop(pq)

        if node in visited:
            continue
        visited.add(node)

        for neighbor in graph.neighbors(node):
            if neighbor not in visited:
                edge_data = graph.get_edge_data(node, neighbor)
                new_cost = cost + edge_data['weight']

                if new_cost < distances[neighbor]:
                    distances[neighbor] = new_cost
                    predecessors[neighbor] = node
                    heapq.heappush(pq, (new_cost, neighbor, node))

    return distances, predecessors

def build_routing_table(start_node, predecessors):
    """Builds a routing table from Dijkstra's predecessors map."""
    table = {}
    for dest in predecessors:
        if dest == start_node:
            table[dest] = {'next_hop': '-', 'cost': 0}
            continue

        if predecessors[dest] is None:
            table[dest] = {'next_hop': '-', 'cost': float('inf')}
            continue

        curr = dest
        while predecessors[curr] is not None and predecessors[curr] != start_node:
            curr = predecessors[curr]

        table[dest] = {'next_hop': curr, 'cost': 0}

    return table

def simulate_is_is():
    """Simulates the IS-IS protocol (using Dijkstra)."""

    print("--- Simulating IS-IS (Link-State / Dijkstra) ---")

    G = nx.Graph()
    edges = [
        ('R1', 'R2', 10), ('R1', 'R3', 5),
        ('R2', 'R3', 2), ('R2', 'R4', 1),
        ('R3', 'R2', 2), ('R3', 'R4', 9), ('R3', 'R5', 2),
        ('R4', 'R5', 4),
        ('R5', 'R1', 7)
    ]
    for u, v, w in edges:
        G.add_edge(u, v, weight=w)

    print(f"Network Nodes: {list(G.nodes)}")
    print(f"Network Links (with metrics): {edges}\n")

    link_state_database = G
    print("Step 1: Link-State PDU (LSP) flooding simulated.")
    print("All routers now have a complete map (LSDB) of the network.\n")

    pos = nx.spring_layout(G)
    draw_graph_with_costs(G, pos, "IS-IS Network Topology (Link Metrics)")

    all_routing_tables = {}

    print("Step 2: Each router runs Dijkstra's algorithm (IS-IS uses SPF).\n")

    for router_name in G.nodes:
        print(f"--- Router {router_name} Calculations ---")

        distances, predecessors = dijkstra(link_state_database, router_name)

        routing_table = build_routing_table(router_name, predecessors)
        for dest in routing_table:
            routing_table[dest]['cost'] = distances[dest]

        all_routing_tables[router_name] = routing_table

        print(f"Routing Table for Router {router_name}:")
        print(f"  {'Destination':<12} | {'Next Hop':<10} | {'Total Cost':<10}")
        print("  " + "-"*40)
        for dest, info in sorted(routing_table.items()):
            print(f"  {dest:<12} | {info['next_hop']:<10} | {info['cost']:<10}")
        print()

if __name__ == "__main__":
    simulate_is_is()

--- Simulating IS-IS (Link-State / Dijkstra) ---
Network Nodes: ['R1', 'R2', 'R3', 'R4', 'R5']
Network Links (with metrics): [('R1', 'R2', 10), ('R1', 'R3', 5), ('R2', 'R3', 2), ('R2', 'R4', 1), ('R3', 'R2', 2), ('R3', 'R4', 9), ('R3', 'R5', 2), ('R4', 'R5', 4), ('R5', 'R1', 7)]

Step 1: Link-State PDU (LSP) flooding simulated.
All routers now have a complete map (LSDB) of the network.


*** Graph saved to isis_topology.png ***
Step 2: Each router runs Dijkstra's algorithm (IS-IS uses SPF).

--- Router R1 Calculations ---
Routing Table for Router R1:
  Destination  | Next Hop   | Total Cost
  ----------------------------------------
  R1           | -          | 0         
  R2           | R3         | 7         
  R3           | R3         | 5         
  R4           | R3         | 8         
  R5           | R5         | 7         

--- Router R2 Calculations ---
Routing Table for Router R2:
  Destination  | Next Hop   | Total Cost
  ----------------------------------------
  R1     

# **OSPF**

In [3]:
import networkx as nx
import matplotlib
matplotlib.use('Agg') # Use a non-GUI backend
import matplotlib.pyplot as plt
import heapq

def draw_graph_with_costs(graph, pos, title):
    """Helper function to draw the network graph with link costs."""
    plt.figure(figsize=(12, 8))
    labels = nx.get_edge_attributes(graph, 'weight')
    nx.draw(graph, pos, with_labels=True, node_color='skyblue', node_size=2500, font_size=16, font_weight='bold')
    nx.draw_networkx_edge_labels(graph, pos, edge_labels=labels, font_color='darkred')
    plt.title(title)

    filename = "ospf_topology.png"
    plt.savefig(filename)
    print(f"\n*** Graph saved to {filename} ***")
    plt.close()

def draw_spt(graph, spt_edges, pos, router_name, title):
    """Helper function to draw a specific router's Shortest Path Tree."""
    plt.figure(figsize=(12, 8))

    nx.draw(graph, pos, with_labels=True, node_color='gray', node_size=2000, font_size=14, alpha=0.3)
    labels = nx.get_edge_attributes(graph, 'weight')
    nx.draw_networkx_edge_labels(graph, pos, edge_labels=labels, font_color='gray', alpha=0.3)

    spt_nodes = set([router_name])
    for u, v in spt_edges:
        spt_nodes.add(u)
        spt_nodes.add(v)

    nx.draw_networkx_nodes(graph, pos, nodelist=spt_nodes, node_color='skyblue', node_size=2500)
    nx.draw_networkx_labels(graph, pos, font_size=16, font_weight='bold')
    nx.draw_networkx_edges(graph, pos, edgelist=spt_edges, edge_color='blue', width=2.5)

    nx.draw_networkx_nodes(graph, pos, nodelist=[router_name], node_color='tomato', node_size=3000)

    plt.title(title)

    filename = f"ospf_spt_{router_name}.png"
    plt.savefig(filename)
    print(f"*** SPT graph saved to {filename} ***")
    plt.close()

def dijkstra(graph, start_node):
    """Implements Dijkstra's algorithm to find shortest paths."""

    pq = [(0, start_node, None)]
    distances = {node: float('inf') for node in graph.nodes}
    distances[start_node] = 0
    predecessors = {node: None for node in graph.nodes}
    spt_edges = []
    visited = set()

    while pq:
        cost, node, prev = heapq.heappop(pq)

        if node in visited:
            continue

        visited.add(node)

        if prev is not None:
            spt_edges.append((prev, node))

        for neighbor in graph.neighbors(node):
            if neighbor not in visited:
                edge_data = graph.get_edge_data(node, neighbor)
                new_cost = cost + edge_data['weight']

                if new_cost < distances[neighbor]:
                    distances[neighbor] = new_cost
                    predecessors[neighbor] = node
                    heapq.heappush(pq, (new_cost, neighbor, node))

    return distances, predecessors, spt_edges

def build_routing_table(start_node, predecessors):
    """Builds a routing table from Dijkstra's predecessors map."""
    table = {}
    for dest in predecessors:
        if dest == start_node:
            table[dest] = {'next_hop': '-', 'cost': 0}
            continue

        if predecessors[dest] is None:
            table[dest] = {'next_hop': '-', 'cost': float('inf')}
            continue

        curr = dest
        while predecessors[curr] is not None and predecessors[curr] != start_node:
            curr = predecessors[curr]

        table[dest] = {'next_hop': curr, 'cost': 0}

    return table


def simulate_ospf():
    """Simulates the Open Shortest Path First (OSPF) protocol."""

    print("--- Simulating OSPF (Dijkstra) ---")

    G = nx.Graph()
    edges = [
        ('A', 'B', 5), ('A', 'C', 4), ('A', 'D', 2),
        ('B', 'C', 3), ('B', 'F', 5),
        ('C', 'D', 2), ('C', 'E', 4),
        ('D', 'E', 7),
        ('E', 'F', 6)
    ]
    for u, v, w in edges:
        G.add_edge(u, v, weight=w)

    print(f"Network Nodes: {list(G.nodes)}")
    print(f"Network Links (with costs): {edges}\n")

    link_state_database = G
    print("Step 1: Link-State Advertisement (LSA) flooding simulated.")
    print("All routers now have a complete map (LSDB) of the network.\n")

    pos = nx.spring_layout(G)
    draw_graph_with_costs(G, pos, "OSPF Network Topology (Link Costs)")

    all_routing_tables = {}

    print("Step 2: Each router runs Dijkstra's algorithm to build its SPT.\n")

    for router_name in G.nodes:
        print(f"--- Router {router_name} Calculations ---")

        distances, predecessors, spt_edges = dijkstra(link_state_database, router_name)

        routing_table = build_routing_table(router_name, predecessors)
        for dest in routing_table:
            routing_table[dest]['cost'] = distances[dest]

        all_routing_tables[router_name] = routing_table

        print(f"Shortest Path Tree (SPT) for Router {router_name} (Edges): {spt_edges}")
        print(f"Routing Table for Router {router_name}:")
        print(f"  {'Destination':<12} | {'Next Hop':<10} | {'Total Cost':<10}")
        print("  " + "-"*40)
        for dest, info in sorted(routing_table.items()):
            print(f"  {dest:<12} | {info['next_hop']:<10} | {info['cost']:<10}")
        print()

        draw_spt(G, spt_edges, pos, router_name, f"Shortest Path Tree (SPT) for Router {router_name}")

if __name__ == "__main__":
    simulate_ospf()

--- Simulating OSPF (Dijkstra) ---
Network Nodes: ['A', 'B', 'C', 'D', 'F', 'E']
Network Links (with costs): [('A', 'B', 5), ('A', 'C', 4), ('A', 'D', 2), ('B', 'C', 3), ('B', 'F', 5), ('C', 'D', 2), ('C', 'E', 4), ('D', 'E', 7), ('E', 'F', 6)]

Step 1: Link-State Advertisement (LSA) flooding simulated.
All routers now have a complete map (LSDB) of the network.


*** Graph saved to ospf_topology.png ***
Step 2: Each router runs Dijkstra's algorithm to build its SPT.

--- Router A Calculations ---
Shortest Path Tree (SPT) for Router A (Edges): [('A', 'D'), ('A', 'C'), ('A', 'B'), ('C', 'E'), ('B', 'F')]
Routing Table for Router A:
  Destination  | Next Hop   | Total Cost
  ----------------------------------------
  A            | -          | 0         
  B            | B          | 5         
  C            | C          | 4         
  D            | D          | 2         
  E            | C          | 8         
  F            | B          | 10        

*** SPT graph saved to ospf_spt

# **RIP**

In [4]:
import networkx as nx
import matplotlib
matplotlib.use('Agg') # Use a non-GUI backend
import matplotlib.pyplot as plt
import time

def draw_graph(graph, labels, pos, title):
    """Helper function to draw the network graph and save to file."""
    plt.figure(figsize=(10, 6))
    nx.draw(graph, pos, with_labels=True, node_color='lightblue', node_size=2000, font_size=16, font_weight='bold')
    nx.draw_networkx_edge_labels(graph, pos, edge_labels=labels, font_color='red')
    plt.title(title)

    filename = "rip_topology.png"
    plt.savefig(filename)
    print(f"\n*** Graph saved to {filename} ***")
    plt.close()

def simulate_rip():
    """Simulates the Routing Information Protocol (RIP)."""

    # 1. Create a network topology
    nodes = ['A', 'B', 'C', 'D', 'E']
    edges = [('A', 'B'), ('A', 'C'), ('B', 'C'), ('C', 'D'), ('D', 'E')]

    network = {node: {} for node in nodes}
    for u, v in edges:
        network[u][v] = 1
        network[v][u] = 1

    tables = {node: {} for node in nodes}
    for node in nodes:
        tables[node][node] = {'next_hop': node, 'cost': 0}
        for neighbor in network[node]:
            tables[node][neighbor] = {'next_hop': neighbor, 'cost': 1}

    print("--- Simulating RIP (Bellman-Ford) ---")
    print(f"Network Nodes: {nodes}")
    print(f"Network Links (all cost 1): {edges}\n")

    # 3. Simulate periodic routing updates until convergence
    round_num = 0
    while True:
        round_num += 1
        print(f"--- ROUND {round_num} ---")

        changed = False
        tables_snapshot = {node: table.copy() for node, table in tables.items()}

        for u in nodes:
            for v in network[u]:
                neighbor_table = tables_snapshot[v]

                for dest in neighbor_table:
                    cost_to_neighbor = network[u][v]
                    cost_from_neighbor_to_dest = neighbor_table[dest]['cost']
                    new_cost = cost_to_neighbor + cost_from_neighbor_to_dest

                    if new_cost > 15:
                        new_cost = 16

                    if dest not in tables[u] or new_cost < tables[u][dest]['cost']:
                        tables[u][dest] = {'next_hop': v, 'cost': new_cost}
                        changed = True
                    elif tables[u].get(dest) and tables[u][dest]['next_hop'] == v and new_cost > tables[u][dest]['cost']:
                        tables[u][dest]['cost'] = min(new_cost, 16)
                        changed = True

        # Print tables for this round
        for node in nodes:
            print(f"Router {node} Table (Round {round_num}):")
            for dest, info in sorted(tables[node].items()):
                print(f"  -> Dest: {dest}, Next Hop: {info['next_hop']}, Cost: {info['cost']}")

        print("-" * 20)

        if not changed:
            print(f"\n*** CONVERGENCE REACHED in {round_num} rounds. ***\n")
            break

        if round_num > 10:
            print("Reached max rounds, stopping.")
            break

        time.sleep(1)

    # 4. Display final routing tables
    print("--- FINAL CONVERGED ROUTING TABLES ---")
    for node in nodes:
        print(f"Router {node}'s Final Table:")
        for dest, info in sorted(tables[node].items()):
            print(f"  -> Dest: {dest}, Next Hop: {info['next_hop']}, Cost: {info['cost']}")
        print()

    # Visualization
    G = nx.Graph()
    G.add_edges_from(edges)
    pos = nx.spring_layout(G)
    labels = {(u, v): 1 for u, v in edges}
    draw_graph(G, labels, pos, "RIP Network Topology (All Link Costs = 1)")

if __name__ == "__main__":
    simulate_rip()

--- Simulating RIP (Bellman-Ford) ---
Network Nodes: ['A', 'B', 'C', 'D', 'E']
Network Links (all cost 1): [('A', 'B'), ('A', 'C'), ('B', 'C'), ('C', 'D'), ('D', 'E')]

--- ROUND 1 ---
Router A Table (Round 1):
  -> Dest: A, Next Hop: A, Cost: 0
  -> Dest: B, Next Hop: B, Cost: 1
  -> Dest: C, Next Hop: C, Cost: 1
  -> Dest: D, Next Hop: C, Cost: 2
Router B Table (Round 1):
  -> Dest: A, Next Hop: A, Cost: 1
  -> Dest: B, Next Hop: B, Cost: 0
  -> Dest: C, Next Hop: C, Cost: 1
  -> Dest: D, Next Hop: C, Cost: 2
Router C Table (Round 1):
  -> Dest: A, Next Hop: A, Cost: 1
  -> Dest: B, Next Hop: B, Cost: 1
  -> Dest: C, Next Hop: C, Cost: 0
  -> Dest: D, Next Hop: D, Cost: 1
  -> Dest: E, Next Hop: D, Cost: 2
Router D Table (Round 1):
  -> Dest: A, Next Hop: C, Cost: 2
  -> Dest: B, Next Hop: C, Cost: 2
  -> Dest: C, Next Hop: C, Cost: 1
  -> Dest: D, Next Hop: D, Cost: 0
  -> Dest: E, Next Hop: E, Cost: 1
Router E Table (Round 1):
  -> Dest: C, Next Hop: D, Cost: 2
  -> Dest: D, Next H